# Chapter 12. 적용성 함수자와 순회 가능 함수자

##12.1. 모나드의 일반화

* monad의 경우 기본 인터페이스는 flatMap과 Unit 인데, 이를 기본 인터페이스로 두지 않고 다른 형태를 기본 인터페이스로 둔다면 새로운 추상 형태를 얻게 된다. 이러한 새 추상을 적용성 함수자라고 한다.

### examples 1

```scala
trait Applicative[F[_]] extends Functor[F] {
    // 기본 수단 조합기들
    def map2[A,B,C](fa: F[A], fb: F[B])(f: (A, B) => C): F[C]
    def unit[A](a: => A): F[A]
    
    // 파생된 조합기들
    def map[A, B](fa: F[A])(f: A => B): F[B] = map2(fa, unit(()))((a, _) => f(a))
    
    def traverse[A,B](as: List[A])(f:A => F[B]): F[List[B]]
        as.foldRight(unit(List[B]()))((a, fbs) => map2(f(a), fbs)(_ :: _))
}
```

### examples 2

```scala
trait Monad[F[_]] extends Applicative[F] {
    // 모나드가 되기 위해서는 반드시 unit이 구현되어야 하고 flatMap 또는 join과 map을 재정의 해야 한다.
    def flatMap[A,B](fa: F[A])(f: A => F[B]) : F[B] = join(map(fa)(f))
    def join[A](ffa: F[F[A]]): F[A] = flatMap(ffa)(fa => fa)
    
```

* 이미 모나드인 자료 형식이라면 적용성 함수자 역시 될 수 있다.


## 10.3. 모나드와 적용성 함수자의 차이

* 모나드를 규정하는 연산 집합
   - unit, flatMap
   - unit, compose
   - unit, map, join
   
* 적용성 Option과 Option 모나드

``` scala
    val o: Option[String] =
        F.map2(depts.get("Alice"), salaries.get("Alice"))(
            (dept, salary) => s"Alice in $dept makes $salary per year"
        )
        
    var o: Option[String] = 
        idsByName.get('Bob').flatMap { id =>
            F.map2(depts.get(id), salaries.get(id))(
                (dept, salary) => s"Bob in $dept makes $salary per year"
             
        )}
```

1. Applicative에서는 계산의 구조가 고정된 반면 monad에서는 이전 계산의 결과가 그 다음 계산에 영향을 미칠 수 있다.

``` scala
    val row: Parser[Row] = F.map2(d, temp)(Row(_,_))

    val rows: Parser[List[Row]] = F.flatMap(header) { row => row.sep("\n") } 
```
2. 열들이 순서대로 나열되어 있다면 Applicative, 열들의 순서 정보가 없고 파일을 통해 순서 정보를 추출해야 한다면 monad.

- Applicative 계산의 구조는 고정되어 있으며, 차례대로 적용한다.
- monad는 문맥 의존 문법, applicative는 문맥 자유 문법
- monad는 코드를 미리 고정 시켜놓는게 아니라 해석 시점에서 코드를 생성할 수 있다.


## 12.4. 적용성 함수의 장점

- 최소한의 가정 만으로도 구현이 가능하다.
- Parser의 경우 좀 더 많은 일을 할 수 있는 Parser 개발이 가능해진다.
- 합성이 가능하다. (모나드는 X)

### 12.4.1 모든 적용성 함수자가 모나드는 아니다

- 적용성 스트림
    - map2와 unit은 정의할 수 있지만 flatMap은 정의할 수 없다.


- 유효성 점검 용 Either 객체
    - Either Monad를 만들기 위해 flatMap을 적용할 경우 한번에 하나씩의 오류만 리턴된다. (flatMap에서는 오류 발생시 그 이후 동작은 무식되기 때문)
    - ex) 사용자가 5개의 오류를 고치기 위해서는 5번 실행을 시켜야함.
    

## 12.5. 적용성 함수자 법칙

### 12.5.1 왼쪽, 오른쪽 항등법칙

```scala
map2(unit(()), fa)((_,a) => a) == fa
map2(fa, unit(()))((a,_) => a) == fa
```

### 12.5.2 결합법칙

``` scala
product(product(fa,fb),fc) == map(product(fa, product(fb,fc)))(assoc)
```

### 12.5.3. 곱의 자연성 법칙

map2로 값들을 결합하기 전에 변환을 적용하든지 결합한 후에 변환을 적용하던지 그 결과는 동일 하다.

## 12.6. 순회 가능 함수자

* traverse 함수와 sequence 함수가 flatMap에 의존하지 않아도 구현할 수 있기 때문에 적용성 함수로 구현이 가능하다.
* 만약 적용성 함수 추상 인터페이스에서 List나 Tree 같은 순회 가능한 자료형식이 파라미터로 들어오면 어떻게 해야 할까?

```scala
trait Traverse[F[_]]{
    def traverse[G[_]:Applicative,A,B](fa: F[A])(f: A=> G[B]): G[F[B]] = sequence(map(fa)(f))
    def sequence[G_]:Applicative,A](fga: F[G[A]]): G[F[A]] = traverse(fga)(ga=>ga)
```
- F[G[A]]를 받아서 F와 G의 순서를 바꾼다.
- ex) List[Option[A]] = > Option[List[A]] : List 요소 중 하나라도 None이면 None을 돌려주고 그렇지 않으면 Some 결과를 돌려준다.
- ex) Tree[Option[A]] => Option[Tree[A]] : Tree 요소 중 하나라도 None이면 None을 돌려주고 그렇지 않으면 Some 결과를 돌려준다.
- ex) Map[K, Par[A]] => Par[Map[K,A]] : Map에 있는 모든 값을 병렬로 계산해 준다.

위와 같이 함으로써 원래 구조를 보존 할 수 있다. 모노이드의 경우 원래 구조는 폐기

## 12.7. Traverse의 용도